In [1]:
####################################################

#   PART 1: define all useful functions

####################################################


#Some parts were generated with the help of ChatGPT

import os
import cv2
import numpy as np
import shutil
import json
from matplotlib import pyplot as plt

def get_image_paths(list_of_folders):
    
    image_paths = []

    for folder in list_of_folders:
        for img in os.listdir(folder):
            if img.endswith(('.jpg','jpeg','JPG','.png','.webp','tif')):
                image_paths.append(os.path.join(folder,img))
    
    return image_paths

def create_output_folders(list_of_folders, prefix='output'):
    
    if type(list_of_folders) is str:
        list_of_folders = [list_of_folders]
    
    if prefix is None:
        prefix = ''
    
    for folder in list_of_folders:
        try:
            os.makedirs(os.path.join(prefix,folder))
        except Exception as e:
            errnum = e.args[0]
            if errnum == 17:
                pass
            else:
                print(e)
    return


def concatenate_images_horizontally(images):
    # Get the maximum height among all images
    max_height = max(img.shape[0] for img in images)

    # Resize images to have the same height
    resized_images = [cv2.resize(img, (int(img.shape[1] * max_height / img.shape[0]), max_height)) for img in images]

    # Concatenate images horizontally
    concatenated_image = np.hstack(resized_images)

    return concatenated_image


def normalize_gradients(grad_mag, grad_dir):
    
    grad_dir = grad_dir%(np.pi/2)
    
    grad_dir = (grad_dir < np.pi/4)*grad_dir + (grad_dir >= np.pi/4)*(np.pi/2 - grad_dir)
    
    grad_mag /= (grad_dir < np.arctan(1/2))*4/np.cos(grad_dir) + (grad_dir >= np.arctan(1/2))*6/(np.cos(grad_dir)+np.sin(grad_dir))
    
    return grad_mag


def remove_border(array_2d, frac=0.01):
    
    height, width = array_2d.shape
    
    remove_h = int(height*frac)
    remove_w = int(width*frac)
    
    new_array = 0*array_2d
    new_array[remove_h:-remove_h,remove_w:-remove_w] = array_2d[remove_h:-remove_h,remove_w:-remove_w]
    
    return new_array


def compute_edges(image, convert=1):
    
    if type(image) is str:
        image = cv2.imread(image)
        

    height, width, _ = image.shape
    factor = np.sqrt(height*width)/512
    image = cv2.resize(image, (int(width//factor),int(height//factor)))
    
    
    # Convert the image to grayscale
    if convert:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)/255
        #gray_image = cv2.GaussianBlur(gray_image, (9,9), 0) 
    else:
        gray_image = image

    # Compute gradients using Sobel operators
    gradient_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)

    # Compute gradient magnitude and direction
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_direction = np.arctan2(gradient_y, gradient_x)
    
    edge_magnitude = normalize_gradients(gradient_magnitude, gradient_direction)
    
    edge_direction = -((gradient_direction%np.pi)-np.pi/2)
    
    
    edge_magnitude = remove_border(edge_magnitude,0.05)
    
    
    return edge_magnitude, edge_direction, image


def binary_array_top_50_percent(array):
    # Flatten the 2D array and sort its elements in descending order
    flattened_array = np.sort(array.flatten())[::-1]

    # Calculate the threshold sum for the top 50% of the original array
    threshold_sum = np.sum(array) * 0.5

    # Initialize variables for cumulative sum and index
    cumulative_sum = 0
    index = 0

    # Iterate through sorted values until cumulative sum exceeds threshold
    while cumulative_sum < threshold_sum:
        cumulative_sum += flattened_array[index]
        index += 1

    # Create a binary array with the same shape as the original array
    binary_array = np.zeros_like(array)

    # Set the top values up to the index to 1 in the binary array
    binary_array[array >= flattened_array[index]] = 1

    return binary_array


def mag_dir_to_img(edge_mag, edge_dir, cmap=0):
    
    #edge_mag = (edge_mag - np.min(edge_mag))/(np.max(edge_mag)-np.min(edge_mag)) REMOVED --> previous results to re run
    
    if cmap==0: 
        
        colors = [(0,255,0),(0,0,255),(255,255,255),(255,0,0),(0,255,0)]
        values = [-np.pi/2,-np.pi/4,0,np.pi/4,np.pi/2+0.1]
    
    
    edge_mag = np.stack([edge_mag, edge_mag, edge_mag],2)
    edge_dir = np.stack([edge_dir, edge_dir, edge_dir],2)
    
    h, w, c = edge_mag.shape
    final_image = np.zeros((h,w,c))
    
    colormaps = [np.zeros_like(final_image)+np.array(c) for c in colors]
    
    for i in range(len(values)-1):
        
        mix = (edge_dir - values[i])/(values[i+1]-values[i])
        
        final_image += ((values[i] <= edge_dir) & (edge_dir < values[i+1]))*((1-mix)*colormaps[i]+mix*colormaps[i+1])
    
    binary_edge = binary_array_top_50_percent(edge_mag)
    exag_image = np.rint(final_image*binary_edge).astype('uint8')
    
    final_image *= edge_mag
    
    return np.rint(final_image).astype('uint8'), exag_image


def define_colormap_for_histograms(start_value = -np.pi/2, end_value = np.pi/2):

    # Create a 2D array with linearly spaced values for each row
    edge_dir = np.array([np.linspace(start_value, end_value, 2000) for _ in range(10)])
    edge_mag = np.ones_like(edge_dir)

    return mag_dir_to_img(edge_mag, edge_dir)


def compute_histogram(edge_map, direction_map, angle_bin=5):
    
    flat_dir = np.rad2deg(direction_map.flatten())
    flat_mag = edge_map.flatten()

    angles = np.arange(-90-angle_bin/2,90+angle_bin/2 +0.1,angle_bin)

    hist, bins = np.histogram(flat_dir, bins=angles, weights=flat_mag/len(flat_mag)) #/len(flat_mag) to remove the influence of number of pixels
    hist[0] += hist[-1]
    hist = hist[:-1] #to avoid counting twice some values
    
    density_hist = hist/np.sum(hist)
    
    return hist, density_hist, bins


def plot_histogram(hist, bins, col_map, start_value = -np.pi/2, end_value = np.pi/2):
    
    h = np.append(hist, hist[0])
    x = (bins[:-1] + bins[1:]) / 2

    fig = plt.figure(figsize=(6,6))
    plt.plot(x, h,color='black',linewidth=0.5)

    xx = np.rad2deg(np.linspace(start_value, end_value, 2000))
    for i in range(len(xx)-1):
        for j in range(len(x)-1):
            if x[j] <= xx[i] < x[j+1]:
                h1 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i]-x[j])
                h2 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i+1]-x[j])
                #plt.plot([xx[i],xx[i+1]],[h1,h2])
                plt.fill_between([xx[i],xx[i+1]],[h1,h2],color=tuple(col_map[0,i,:]/255))

    return plt.gcf()


def plot_circular_histogram(hist, bins, direction, magnitude, col_map, start_value = -np.pi/2, end_value = np.pi/2):
    
    xxr = np.linspace(start_value, end_value+0.001, 2000)
    xx = np.rad2deg(xxr)
    
    hist = hist/np.sum(hist)
    h = np.append(hist, hist[0])
    x = (bins[:-1] + bins[1:]) / 2

    R = np.max(h)
    fig = plt.figure(figsize=(6,6))
    plt.axis('off')
    circle = plt.Circle((0,0),R,facecolor='none',edgecolor='black')
    plt.gca().add_patch(circle)
    ax = plt.gca()

    angs = [np.pi/8,3*np.pi/8,-np.pi/8,-3*np.pi/8]
    for a in angs:
        ax.plot([-R*np.cos(a), R*np.cos(a)], [-R*np.sin(a), R*np.sin(a)], color='grey', linestyle='--',linewidth=0.5)

    for i in range(len(xx)-1):
        for j in range(len(x)-1):
            if x[j] <= xx[i] < x[j+1]:
                h1 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i]-x[j])
                h2 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i+1]-x[j])

                plt.fill([0, h1*np.cos(xxr[i]), h2*np.cos(xxr[i+1]), 0],[0, h1*np.sin(xxr[i]), h2*np.sin(xxr[i+1]), 0], color=tuple(col_map[0,i,:]/255))
                plt.fill([0, -h1*np.cos(xxr[i]), -h2*np.cos(xxr[i+1]), 0],[0, -h1*np.sin(xxr[i]), -h2*np.sin(xxr[i+1]), 0], color=tuple(col_map[0,i,:]/255))

                plt.plot([h1*np.cos(xxr[i]), h2*np.cos(xxr[i+1])],[h1*np.sin(xxr[i]), h2*np.sin(xxr[i+1])],color='black',linewidth=0.5)
                plt.plot([-h1*np.cos(xxr[i]), -h2*np.cos(xxr[i+1])],[-h1*np.sin(xxr[i]), -h2*np.sin(xxr[i+1])],color='black',linewidth=0.5)


    angs = [-3*np.pi/8,-np.pi/8,np.pi/8,3*np.pi/8]

    prop = []

    for i in range(len(angs)-1):
        p = np.sum(np.extract((angs[i] <= direction) & (direction < angs[i+1]),magnitude))/np.sum(magnitude)
        prop.append(round(p,3))
    prop.append(round(1-np.sum(prop),3))

    for i in range(4):
        xt = 1.2*R*np.cos(angs[i]+np.pi/8)
        yt = 1.1*R*np.sin(angs[i]+np.pi/8)
        ax.text(xt, yt, str(round(100*prop[i],3))+'%', ha='center',fontsize=15)
    
    mag_pp = round(np.sum(magnitude/np.sum(np.ones_like(magnitude))),4)
    radius = round(R,4)
    ax.text(-R,0.8*R, f'Mag_pp: {mag_pp}\n\nRadius: {radius}',ha='center',fontsize=15)
    
    return plt.gcf(), mag_pp, radius


def plot_black_circular_histogram(hist, bins, direction, magnitude, col_map, start_value = -np.pi/2, end_value = np.pi/2):
    
    xxr = np.linspace(start_value, end_value+0.001, 2000)
    xx = np.rad2deg(xxr)
    
    hist = hist/np.sum(hist)
    h = np.append(hist, hist[0])
    x = (bins[:-1] + bins[1:]) / 2

    R = np.max(h)
    fig = plt.figure(figsize=(6,6))
    fig.patch.set_facecolor('black')
    plt.axis('off')
    circle = plt.Circle((0,0),R,facecolor='none',edgecolor='none') #A laisser pour avoir bonne échelle !!!
    plt.gca().add_patch(circle)
    ax = plt.gca()

    for i in range(len(xx)-1):
        for j in range(len(x)-1):
            if x[j] <= xx[i] < x[j+1]:
                h1 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i]-x[j])
                h2 = h[j]+(h[j]-h[j+1])/(x[j]-x[j+1])*(xx[i+1]-x[j])

                plt.fill([0, h1*np.cos(xxr[i]), h2*np.cos(xxr[i+1]), 0],[0, h1*np.sin(xxr[i]), h2*np.sin(xxr[i+1]), 0], color=tuple(col_map[0,i,:]/255))
                plt.fill([0, -h1*np.cos(xxr[i]), -h2*np.cos(xxr[i+1]), 0],[0, -h1*np.sin(xxr[i]), -h2*np.sin(xxr[i+1]), 0], color=tuple(col_map[0,i,:]/255))
    
    mag_pp = round(np.sum(magnitude/np.sum(np.ones_like(magnitude))),4)
    radius = round(R,4)
    
    return plt.gcf(), mag_pp, radius



def average_amount_of_magnitude(edge_map):
    return np.mean(edge_map)

def proportion_of_motion_between_angles(min_angle, max_angle, edge_map, direction_map):
    
    if min_angle > max_angle:
        return 1-proportion_of_motion_between_angles(max_angle, min_angle, edge_map, direction_map)
    
    direction_map = np.rad2deg(direction_map)
    
    return np.sum(np.extract((min_angle <= direction_map) & (direction_map < max_angle),edge_map))/np.sum(edge_map)


def save_to_json(out_path, key, value):

    if out_path[-5:] != '.json':
        json_file_path = out_path+'.json'
    else:
        json_file_path = out_path

    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as file:
            data = json.load(file)
    else:
        data = dict()
    
    data[key] = value

    with open(json_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    return



In [2]:
####################################################

#   PART 2: run the analysis

####################################################


folders = ["my_folder_of_images"] #list of folders containing images to analyze
out_prefix = 'out_edges' # newly created parent output folder, e.g. out_edges/my_folder_of_images


################


angle_bin = 5

image_paths = get_image_paths(folders)

create_output_folders(folders,prefix=out_prefix)

col_map_for_hist = define_colormap_for_histograms()[0]

count = 0

for image_path in image_paths:
    
    count += 1
    print(image_path, count)
    
    img_name, img_ext = os.path.splitext(os.path.basename(image_path))
    out_folder = os.path.splitext(os.path.join(out_prefix, image_path))[0]
    
    if os.path.isdir(out_folder):
        continue
        
    create_output_folders(out_folder,prefix=None)
    out_path = os.path.join(out_folder, img_name)
    
    
    edge_map, direction_map, image = compute_edges(image_path)
    edge_img, exag_img = mag_dir_to_img(edge_map, direction_map)
    
    cv2.imwrite(out_path+img_ext, image)
    cv2.imwrite(out_path+'-edges_dir'+img_ext, cv2.cvtColor(edge_img, cv2.COLOR_RGB2BGR))
    cv2.imwrite(out_path+'-edges_dir_50'+img_ext, cv2.cvtColor(exag_img, cv2.COLOR_RGB2BGR))
    save_to_json(out_path, key='image_path', value=out_path+img_ext)
    save_to_json(out_path, key='image_edges_dir_path', value=out_path+'-edges_dir'+img_ext)
    save_to_json(out_path, key='image_edges_dir_50_path', value=out_path+'-edges_dir_50'+img_ext)
    
    np.save(out_path+'-edge_map.npy', edge_map)
    np.save(out_path+'-direction_map.npy', direction_map)
    save_to_json(out_path, key='edge_map_path', value=out_path+'-edge_map.npy')
    save_to_json(out_path, key='direction_map_path', value=out_path+'-direction_map.npy')
    
    hist, d_hist, bins = compute_histogram(edge_map, direction_map, angle_bin)
    np.save(f'{out_path}-histogram_angbin_{angle_bin}.npy', hist)
    save_to_json(out_path, key=f'histogram_angbin_{angle_bin}', value=f'{out_path}-histogram_angbin_{angle_bin}.npy')
    np.save(f'{out_path}-histogram_angbin_{angle_bin}_density_1.npy', d_hist)
    save_to_json(out_path, key=f'histogram_angbin_{angle_bin}_density_1', value=f'{out_path}-histogram_angbin_{angle_bin}_density_1.npy')
    
    draw_hist = plot_histogram(hist, bins, col_map_for_hist)
    draw_hist.savefig(f'{out_path}-histogram_angbin_{angle_bin}.png', bbox_inches='tight')
    save_to_json(out_path, key=f'draw_histogram_angbin_{angle_bin}', value=f'{out_path}-histogram_angbin_{angle_bin}.png')
    plt.close(draw_hist)
    draw_hist = plot_histogram(d_hist, bins, col_map_for_hist)
    draw_hist.savefig(f'{out_path}-histogram_angbin_{angle_bin}_density_1.png', bbox_inches='tight')
    save_to_json(out_path, key=f'draw_histogram_angbin_{angle_bin}_density_1', value=f'{out_path}-histogram_angbin_{angle_bin}_density_1.png')
    plt.close(draw_hist)
    
    circ_hist, mag_pp, rad = plot_circular_histogram(hist, bins, direction_map, edge_map, col_map_for_hist)
    circ_hist.savefig(f'{out_path}-circular_histogram_angbin_{angle_bin}.png', bbox_inches='tight')
    save_to_json(out_path, key=f'draw_circular_histogram_angbin_{angle_bin}', value=f'{out_path}-circular_histogram_angbin_{angle_bin}.png')
    save_to_json(out_path, key=f'magnitude_per_pixel', value=mag_pp)
    save_to_json(out_path, key=f'radius', value=rad)
    plt.close(circ_hist)
    
    
    black_circ_hist, _, _ = plot_black_circular_histogram(hist, bins, direction_map, edge_map, col_map_for_hist)
    black_circ_hist.savefig(f'{out_path}-black_circular_histogram_angbin_{angle_bin}.png', bbox_inches='tight')
    save_to_json(out_path, key=f'draw_black_circular_histogram_angbin_{angle_bin}', value=f'{out_path}-black_circular_histogram_angbin_{angle_bin}.png')
    plt.close(black_circ_hist)
    
    
    circ_hist_plot = cv2.imread(f'{out_path}-circular_histogram_angbin_{angle_bin}.png')
    concat = concatenate_images_horizontally([image, cv2.cvtColor(exag_img, cv2.COLOR_RGB2BGR), circ_hist_plot])
    cv2.imwrite(out_path+'-concat.png', concat)
    save_to_json(out_path, key='concat_results_path', value=out_path+'-concat.png')
    
    
    avg_mag = average_amount_of_magnitude(edge_map)
    d = np.rad2deg(np.abs(direction_map))
    diag = np.sum(np.extract((22.5 <= d) & (d < 67.5),edge_map))/np.sum(edge_map)
    
    save_to_json(out_path, key='average_magnitude', value=avg_mag)
    save_to_json(out_path, key='diag_motion', value=diag)
    save_to_json(out_path, key='average_direction', value=np.mean(np.rad2deg(direction_map)))
    save_to_json(out_path, key='average_weighted_direction', value=np.sum(edge_map*np.rad2deg(direction_map))/np.sum(edge_map))
    save_to_json(out_path, key='average_absolute_direction', value=np.mean(np.rad2deg(np.abs(direction_map))))
    save_to_json(out_path, key='average_weighted_absolute_direction', value=np.sum(edge_map*np.rad2deg(np.abs(direction_map)))/np.sum(edge_map))
    

my_folder_of_images/1940_Moderation.jpg 1
my_folder_of_images/1940_Diverse_Parties.jpg 2
my_folder_of_images/1940_Sky_Blue.jpg 3
my_folder_of_images/1940_Little_Accents.jpg 4
my_folder_of_images/1940_Around_the_Circle.jpg 5
